In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient
import requests
import time
import datetime
import json

SHADOW_CLIENT = "A0195028B_2"
HOST_NAME = "a2h9i547jb98l3-ats.iot.us-west-2.amazonaws.com"
ROOT_CA = "/Users/yuxingli/Desktop/IoT/AmazonRootCA1.pem"
PRIVATE_KEY = "/Users/yuxingli/Desktop/IoT/324242c7b5-private.pem.key"
CERT_FILE = "/Users/yuxingli/Desktop/IoT/324242c7b5-certificate.pem.crt"
SHADOW_HANDLER = "A0195028B_2"
matricNo = "A0195028B"
url = "https://github.com/iceberg12/NUS_guest_lecture/blob/master/input/train_FD002.txt?raw=true"
r = requests.get(url)
with open("train_FD002.txt", "wb") as code:
    code.write(r.content)
f = open("train_FD002.txt")
line = f.readlines()


def myShadowUpdateCallback(payload, responseStatus, token):
    print()
    print('UPDATE: $aws/things/' + SHADOW_HANDLER +'/shadow/update/#')
    print("payload = " + payload)
    print("responseStatus = " + responseStatus)
    print("token = " + token)


# Create, configure, and connect a shadow client.
myShadowClient = AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME, 8883)
myShadowClient.configureCredentials(ROOT_CA, PRIVATE_KEY, CERT_FILE)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(5)
myShadowClient.connect()

# Create a programmatic representation of the shadow.
myDeviceShadow = myShadowClient.createShadowHandlerWithName(SHADOW_HANDLER, True)


for i in line:
    x = i.split()
    x[0] = 'FD002_'+x[0]
    y = {"id": x[0], "circle": x[1], "os1": x[2], "os2": x[3], "os3": x[4]}
    for j in range(21):
        y["sensor"+str(j+1)] = x[j+5]
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
    y["timestamp"] = (t)
    y["Matric No"] = matricNo
    myDeviceShadow.shadowUpdate(str(json.dumps(y)), myShadowUpdateCallback, 5)
    time.sleep(0.2)